In [97]:

# Import Required Modules
from flask import Flask, render_template
import pandas as pd
import json
import plotly
import plotly.express as px
import pandas_ta as pta
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import date
from datetime import datetime
from datetime import timedelta, date
from prophet import Prophet
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import mpld3
import matplotlib.pyplot as plt
import warnings


# Create Home Page Route
app = Flask(__name__)

   
df = pd.read_csv("Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.Ticker('BTC-USD').history(period='7y',interval='1d',actions=False).reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
btc_df['date'] = btc_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
btc_df['date'] = pd.to_datetime(btc_df['date'])

df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Unnamed: 0', "wallets", "address", "mined","Open_x","High_x","Low_x","Volume_x","Open_y","High_y","Low_y","Volume_y"])
df

c:\Users\chris\anaconda3\envs\python_data\Lib\site-packages\yfinance\utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.



,date,price
0,2010-09-25,0.100000
1,2010-09-26,0.100000
2,2010-09-27,0.100000
3,2010-09-28,0.100000
4,2010-09-29,0.100000
...,...,...
5162,2024-12-10,96675.429688
5163,2024-12-11,101173.031250
5164,2024-12-12,100043.000000
5165,2024-12-13,101459.257812


In [98]:
mstr_df = yf.Ticker('MSTR').history(period='7y',interval='1d',actions=False).reset_index()
mstr_df = mstr_df.rename(columns={"Date": "date"})
mstr_df['date'] = mstr_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
mstr_df['price'] = mstr_df[['Open', 'High', 'Low', 'Close']].mean(axis=1)
mstr_df = mstr_df.drop(columns=['Volume', 'Open', 'High', 'Low', 'Close'])
mstr_df

c:\Users\chris\anaconda3\envs\python_data\Lib\site-packages\yfinance\utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.



,date,price
0,2017-12-14,13.211000
1,2017-12-15,13.195250
2,2017-12-18,13.371500
3,2017-12-19,13.439000
4,2017-12-20,13.488000
...,...,...
1756,2024-12-09,380.644997
1757,2024-12-10,371.634995
1758,2024-12-11,398.809998
1759,2024-12-12,401.952492


In [99]:
# Convert 'date' columns to datetime
df['date'] = pd.to_datetime(df['date'])
mstr_df['date'] = pd.to_datetime(mstr_df['date'])


# Merge the dataframes on the 'date' column
combined_df = pd.merge(df, mstr_df, on='date', how='outer')
combined_df = combined_df.rename(columns={"price_x": "bitcoin", "price_y": "mstr"})
# Drop rows with any NaN values
combined_df = combined_df.dropna()
# Filter the combined dataframe to only include data after August 2020
combined_df = combined_df[combined_df['date'] > '2020-08-01']
# Create a new column 'mstr_to_bitcoin' by dividing 'mstr' by 'bitcoin'
combined_df['mstr_to_bitcoin'] = combined_df['mstr']*1000 / combined_df['bitcoin']
# Display the combined dataframe
# Calculate moving averages
combined_df['mstr_to_bitcoin_7d'] = combined_df['mstr_to_bitcoin'].rolling(window=7).mean()
combined_df['mstr_to_bitcoin_20d'] = combined_df['mstr_to_bitcoin'].rolling(window=20).mean()
combined_df['mstr_to_bitcoin_50d'] = combined_df['mstr_to_bitcoin'].rolling(window=50).mean()
combined_df['mstr_to_bitcoin_100d'] = combined_df['mstr_to_bitcoin'].rolling(window=100).mean()
combined_df

,date,bitcoin,mstr,mstr_to_bitcoin,mstr_to_bitcoin_7d,mstr_to_bitcoin_20d,mstr_to_bitcoin_50d,mstr_to_bitcoin_100d
3579,2020-08-03,11224.400000,12.372000,1.102242,NaN,NaN,NaN,NaN
3580,2020-08-04,11184.700000,12.291750,1.098979,NaN,NaN,NaN,NaN
3581,2020-08-05,11735.100000,12.414250,1.057873,NaN,NaN,NaN,NaN
3582,2020-08-06,11757.100000,12.330000,1.048728,NaN,NaN,NaN,NaN
3583,2020-08-07,11592.000000,12.318500,1.062673,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5166,2024-12-09,97432.718750,380.644997,3.906747,4.014239,4.099398,3.543801,2.938551
5167,2024-12-10,96675.429688,371.634995,3.844151,3.980082,4.110939,3.566992,2.951876
5168,2024-12-11,101173.031250,398.809998,3.941861,3.966842,4.112230,3.591778,2.964918
5169,2024-12-12,100043.000000,401.952492,4.017797,3.983880,4.119866,3.617858,2.978556


In [100]:
fig = px.line(combined_df, x='date', y='mstr_to_bitcoin', title='MSTR to Bitcoin Ratio Over Time')
# fig.add_scatter(x=combined_df['date'], y=combined_df['mstr_to_bitcoin_7d'], mode='lines', name='7 Day MA')
fig.add_scatter(x=combined_df['date'], y=combined_df['mstr_to_bitcoin_20d'], mode='lines', name='20 Day MA')
# fig.add_scatter(x=combined_df['date'], y=combined_df['mstr_to_bitcoin_50d'], mode='lines', name='50 Day MA')
# fig.add_scatter(x=combined_df['date'], y=combined_df['mstr_to_bitcoin_100d'], mode='lines', name='100 Day MA')
fig.add_scatter(x=combined_df['date'], y=combined_df['mstr'], mode='lines', name='MSTR Price', yaxis='y2')

# Update layout to add a secondary y-axis
fig.update_layout(
    yaxis2=dict(
        title="MSTR Price",
        overlaying='y',
        side='right'
    )
)
fig.show()

In [101]:
fig = px.line(combined_df, x='date', y='mstr_to_bitcoin', title='MSTR to Bitcoin Ratio Over Time')
fig.add_scatter(x=combined_df['date'], y=combined_df['mstr_to_bitcoin_20d'], mode='lines', name='20 Day MA')
fig.show()

In [102]:
# Keep only the specified columns
combined_df = combined_df[['date', 'bitcoin', 'mstr','mstr_to_bitcoin', 'mstr_to_bitcoin_20d']]
combined_df['mstr_to_bitcoin_diff'] = combined_df['mstr_to_bitcoin'] - combined_df['mstr_to_bitcoin_20d']
# Calculate the standard deviation of 'mstr_to_bitcoin_diff'
std_dev = combined_df['mstr_to_bitcoin_diff'].std()

# Define the bins based on standard deviation
bins = [-2*std_dev, -std_dev, 0, std_dev, 2*std_dev]

# Create labels for the bins
group_names = ['Severely Oversold', 'Neutral Oversold', 'Neutral Overbought', 'Severely Overbought']
combined_df = combined_df.dropna()
# Create a new column 'std_dev_bucket' with the bin labels
combined_df['std_dev_bucket'] = pd.cut(combined_df['mstr_to_bitcoin_diff'], bins=bins, labels=group_names)
combined_df

C:\Users\chris\AppData\Local\Temp\ipykernel_5616\242159168.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,bitcoin,mstr,mstr_to_bitcoin,mstr_to_bitcoin_20d,mstr_to_bitcoin_diff,std_dev_bucket
3604,2020-08-28,11527.400000,14.788000,1.282856,1.184408,0.098449,Neutral Overbought
3607,2020-08-31,11644.200000,14.591250,1.253092,1.191950,0.061142,Neutral Overbought
3608,2020-09-01,11914.900000,14.698500,1.233623,1.198682,0.034941,Neutral Overbought
3609,2020-09-02,11413.300000,14.935750,1.308627,1.211220,0.097407,Neutral Overbought
3610,2020-09-03,10168.800000,14.642250,1.439919,1.230780,0.209140,Severely Overbought
...,...,...,...,...,...,...,...
5166,2024-12-09,97432.718750,380.644997,3.906747,4.099398,-0.192650,Severely Oversold
5167,2024-12-10,96675.429688,371.634995,3.844151,4.110939,-0.266787,Severely Oversold
5168,2024-12-11,101173.031250,398.809998,3.941861,4.112230,-0.170370,Neutral Oversold
5169,2024-12-12,100043.000000,401.952492,4.017797,4.119866,-0.102069,Neutral Oversold


In [107]:
fig = px.scatter(combined_df, x="date", y="mstr_to_bitcoin", color="std_dev_bucket", 
                 color_discrete_sequence=["orange", "red", "blue", "green"],
                 title="MSTR to Bitcoin Ratio with Standard Deviation Buckets")
fig.add_scatter(x=combined_df['date'], y=combined_df['mstr_to_bitcoin_20d'], mode='lines', name='20 Day MA')
fig.show()